# Preparando o ambiente

In [1]:
from modules.utils import prepare_data_filesystem

prepare_data_filesystem()

[03/04/2024 19:40:35] Tentando criar o diretório data/raw.
[03/04/2024 19:40:35] O diretório data/raw já existe, nada será feito.
[03/04/2024 19:40:35] Tentando criar o diretório data/processed.
[03/04/2024 19:40:35] O diretório data/processed já existe, nada será feito.


# Programando as execuções

In [2]:
from modules.etl.first import bronze_step

bronze_step(query="dengue",languague="pt")

[03/04/2024 19:40:35] FIST STEP (FOR EACH 1 HOUR)
[03/04/2024 19:40:35] Extracting...
[03/04/2024 19:40:35] Construindo os dados a serem enviados na requisição
[03/04/2024 19:40:35] Enviando a requisição
[03/04/2024 19:40:39] Requisição feita com sucesso!!
[03/04/2024 19:40:39] Não há novos artigos no periodo de 2024-04-03T18:00:35 a 2024-04-03T19:00:35
